<a href="https://colab.research.google.com/github/sudip4474657/blender-colab/blob/main/blender_render222222222222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.
* `optix_enable` : Enable OptiX which may boost performance, may be incompatible depending on the version of blender, project and GPU allocated

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more information.

In [1]:
blender_version = '4.3.2' #@param ['2.79b', '2.83.20', '2.93.18', '3.3.21', '3.6.21', '4.2.7', '4.3.2'] {allow-input: false}
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'google_drive' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'axt/corv/corv1.blend' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = True #@param {type: 'boolean'}
start_frame =  190#@param {type: 'integer'}
end_frame =  200#@param {type: 'integer'}
#@markdown ---
download_type = 'gdrive_direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'corv-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'axt/corv/' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}

In [2]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

/content
Current GPU: Tesla T4


In [3]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  google-perftools libgoogle-perftools4 libtcmalloc-minimal4
0 upgraded, 0 newly installed, 3 to remove and 29 not upgraded.
After this operation, 1,817 kB disk space will be freed.
(Reading database ... 126209 files and directories currently installed.)
Removing google-perftools (2.9.1-0ubuntu3) ...
Removing libgoogle-perftools4:amd64 (2.9.1-0ubuntu3) ...
Removing libtcmalloc-minimal4:amd64 (2.9.1-0ubuntu3) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/loc

In [4]:
import shutil
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

Mounted at /drive


In [5]:
!rm -r render
!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/My Drive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")

rm: cannot remove 'render': No such file or directory


In [6]:
# To add your own new version, go to https://ftp.nluug.nl/pub/graphics/blender/release
# and copy the URL of the desired version linux-x64 tar archive
# Then, edit or add the version below and also at the version selection at the top
blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.83.20' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz",
                    '2.93.18' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.18-linux-x64.tar.xz",
                    '3.3.21'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.21-linux-x64.tar.xz",
                    '3.6.21'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.6/blender-3.6.21-linux-x64.tar.xz",
                    '4.2.7'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.2/blender-4.2.7-linux-x64.tar.xz",
                    '4.3.2'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.3/blender-4.3.2-linux-x64.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

--2025-03-27 12:31:19--  https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.3/blender-4.3.2-linux-x64.tar.xz
Resolving ftp.nluug.nl (ftp.nluug.nl)... 145.220.21.40, 2001:67c:6ec:221:145:220:21:40
Connecting to ftp.nluug.nl (ftp.nluug.nl)|145.220.21.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 368852356 (352M) [application/octet-stream]
Saving to: ‘blender-4.3.2-linux-x64.tar.xz’

blender-4.3.2-linux 100%[===================>] 351.76M  19.8MB/s    in 19s     

2025-03-27 12:31:40 (18.2 MB/s) - ‘blender-4.3.2-linux-x64.tar.xz’ saved [368852356/368852356]



In [11]:
import os

# Define the directory and filename
directory = "4.3.2/"
filename = "setgpu.py"
file_path = os.path.join(directory, filename)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Define the Python code
python_code = """import re
import bpy
scene = bpy.context.scene
scene.cycles.device = 'GPU'
prefs = bpy.context.preferences
prefs.addons['cycles'].preferences.get_devices()
cprefs = prefs.addons['cycles'].preferences
print(cprefs)
for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):
    try:
        cprefs.compute_device_type = compute_device_type
        print('Device found:', compute_device_type)
        break
    except TypeError:
        pass
for device in cprefs.devices:
    if not re.match('intel', device.name, re.I):
        print('Activating', device)
        device.use = True
    else:
        device.use = True"""

# Write the code to the file
with open(file_path, "w") as file:
    file.write(python_code)

print(f"File saved at: {file_path}")


File saved at: 4.3.2/setgpu.py


In [12]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (example: Tesla K80) is not supported and you need to switch back to Cycles.")
    renderer = "OPTIX"

In [13]:
%cd /content

!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = '/content/output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name

%cd /content/$blender_version

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"

/content
/content/4.3.2
Blender 4.3.2 (hash 32f5fdce0a0a built 2024-12-17 01:31:14)
Read blend: "/content/render/corv1.blend"
<bpy_struct, CyclesPreferences at 0x7bd2ffa97948>
Device found: CUDA
Activating <bpy_struct, CyclesDeviceSettings("Tesla T4") at 0x7bd2ffadc288>
Activating <bpy_struct, CyclesDeviceSettings("Tesla T4") at 0x7bd2ffadc398>
Fra:190 Mem:283.65M (Peak 628.95M) | Time:00:00.33 | Mem:0.00M, Peak:0.00M | Car, ViewLayer | Synchronizing object | Airstem
Fra:190 Mem:283.67M (Peak 628.95M) | Time:00:00.33 | Mem:0.00M, Peak:0.00M | Car, ViewLayer | Synchronizing object | car
Fra:190 Mem:285.02M (Peak 628.95M) | Time:00:00.33 | Mem:0.00M, Peak:0.00M | Car, ViewLayer | Synchronizing object | Front_Logo.002
Fra:190 Mem:286.37M (Peak 628.95M) | Time:00:00.33 | Mem:0.00M, Peak:0.00M | Car, ViewLayer | Synchronizing object | Cube.003
Fra:190 Mem:286.44M (Peak 628.95M) | Time:00:00.33 | Mem:0.00M, Peak:0.00M | Car, ViewLayer | Synchronizing object | Front_Logo.001
Fra:190 Mem:287.4

In [10]:
%cd /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

/content


## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020-2022 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```